In [70]:
import os
import sys
import pandas as pd
import numpy as np
from rpy2.robjects import packages, pandas2ri, r

PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), '../'))

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
    
from src.data.fitzroy_data import fitzroy, r_to_pandas

In [71]:
fr = fitzroy()
player_data = fr.get_afltables_stats(start_date='2010-01-01')
player_data



|==                                                    |  4% ~2 s remaining

 

 

 

 

 


|===                                                   |  7% ~3 s remaining

 

 

 

 

 


|====                                                  |  9% ~4 s remaining

 

 

 

 

 


|======                                                | 11% ~4 s remaining

 

 

 

 

 


|=======                                               | 13% ~5 s remaining

 

 

 

 

 


|========                                              | 16% ~5 s remaining

 

 

 

 

 


|=========                                             | 18% ~5 s remaining

 

 

 

 

 


|==========                                            | 20% ~5 s remaining

 

 

 

 

 


|============                                          | 22% ~5 s remaining

 

 

 

 

 


|=============                                         | 24% ~5 s remaining

 

 

 

 

 


|==============                                        | 27% ~5 s re

Season,Round,Date,...,Umpire.3,Umpire.4,group_id
2010.000000,'1',14696.000000,...,'Todd Kea...,'',"13,590"
2010.000000,'1',14696.000000,...,'Todd Kea...,'',"13,590"
2010.000000,'1',14696.000000,...,'Todd Kea...,'',"13,590"
2010.000000,'1',14696.000000,...,'Todd Kea...,'',"13,590"
...,...,...,...,...,...,...
2018.000000,'Grand Fi...,17803.000000,...,' Shaun R...,NA_character_,0
2018.000000,'Grand Fi...,17803.000000,...,' Shaun R...,NA_character_,0
2018.000000,'Grand Fi...,17803.000000,...,' Shaun R...,NA_character_,0
2018.000000,'Grand Fi...,17803.000000,...,' Shaun R...,NA_character_,0


In [72]:
# Player data matches have weird round labelling system (lots of strings for finals matches),
# so using round numbers from match_results
match_df = r_to_pandas(r('fitzRoy::match_results'))
player_df = (r_to_pandas(player_data)
             # Some player data venues have trailing spaces
             .assign(venue=lambda x: x['venue'].str.strip())
             # Player data match IDs are wrong for recent years. The easiest way to add correct ones
             # is to graft on the IDs from match_results
             .merge(match_df[['date', 'venue', 'round_number', 'game']], on=['date', 'venue'], how='left')
             # As of 11-10-2018, match_results is still missing finals data from 2018.
             # Joining on date/venue leaves two duplicates played at M.C.G. on 29-4-1986 & 9-8-1986,
             # but that's an acceptable loss of data and easier than munging team names
             .dropna()
             .rename(columns={'season': 'year',
                              'time_on_ground__': 'time_on_ground',
                              'id': 'player_id',
                              'game': 'match_id'})
             .astype({'year': int, 'match_id': int})
             .assign(player_name=lambda x: x['first_name'] + ' ' + x['surname'],
                     id=lambda x: x['player_id'].astype(str) + x['match_id'].astype(str))
             .drop(['first_name', 'surname', 'round', 'local_start_time', 'attendance', 'hq1g', 'hq1b',
                    'hq2g', 'hq2b', 'hq3g', 'hq3b', 'hq4g', 'hq4b', 'aq1g', 'aq1b', 'aq2g', 'aq2b', 'aq3g', 'aq3b',
                    'aq4g', 'aq4b', 'jumper_no_', 'umpire_1', 'umpire_2', 'umpire_3', 'umpire_4',
                    'substitute', 'group_id', 'date', 'venue'], axis=1)
             .drop_duplicates()
             .set_index('id'))
player_df

,year,home_team,home_score,away_team,away_score,player_id,playing_for,kicks,marks,handballs,...,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,time_on_ground,round_number,match_id,player_name
id,,,,,,,,,,,,,,,,,,,,,
1180713585,2010,Fremantle,118,Adelaide,62,11807,Adelaide,5.0,4.0,6.0,...,9.0,0.0,0.0,1.0,0.0,0.0,74,1.0,13585,Tony Armstrong
396813585,2010,Fremantle,118,Adelaide,62,3968,Adelaide,12.0,3.0,5.0,...,14.0,0.0,0.0,6.0,0.0,0.0,98,1.0,13585,Nathan Bock
1171213585,2010,Fremantle,118,Adelaide,62,11712,Adelaide,8.0,4.0,13.0,...,14.0,0.0,0.0,1.0,1.0,0.0,78,1.0,13585,Myke Cook
1170013585,2010,Fremantle,118,Adelaide,62,11700,Adelaide,8.0,3.0,12.0,...,9.0,0.0,1.0,2.0,5.0,0.0,82,1.0,13585,Patrick Dangerfield
8513585,2010,Fremantle,118,Adelaide,62,85,Adelaide,4.0,3.0,13.0,...,12.0,0.0,0.0,4.0,0.0,0.0,91,1.0,13585,Michael Doughty
238113585,2010,Fremantle,118,Adelaide,62,2381,Adelaide,2.0,3.0,10.0,...,9.0,0.0,0.0,1.0,1.0,0.0,74,1.0,13585,Richard Douglas
3413585,2010,Fremantle,118,Adelaide,62,34,Adelaide,16.0,8.0,9.0,...,18.0,1.0,0.0,1.0,2.0,0.0,83,1.0,13585,Tyson Edwards
5113585,2010,Fremantle,118,Adelaide,62,51,Adelaide,7.0,2.0,17.0,...,17.0,0.0,0.0,2.0,0.0,0.0,91,1.0,13585,Simon Goodwin
1153413585,2010,Fremantle,118,Adelaide,62,11534,Adelaide,6.0,3.0,2.0,...,7.0,1.0,0.0,1.0,0.0,1.0,72,1.0,13585,Jonathon Griffin


In [87]:
brownlow_last_year = (player_df[['player_id', 'year', 'brownlow_votes']]
                      .sort_index()
                      .groupby(['player_id', 'year'], group_keys=True)
                      .sum()
                      .groupby(level=0)
                      .shift()
                      .fillna(0)
                      .rename(columns={'brownlow_votes': 'last_year_brownlow_votes'}))

player_df.drop('brownlow_votes', axis=1).merge(brownlow_last_year, on=['player_id', 'year'], how='left')

,year,home_team,home_score,away_team,away_score,player_id,playing_for,kicks,marks,handballs,...,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,time_on_ground,round_number,match_id,player_name,last_year_brownlow_votes
0,2010,Fremantle,118,Adelaide,62,11807,Adelaide,5.0,4.0,6.0,...,0.0,0.0,1.0,0.0,0.0,74,1.0,13585,Tony Armstrong,0.0
1,2010,Fremantle,118,Adelaide,62,3968,Adelaide,12.0,3.0,5.0,...,0.0,0.0,6.0,0.0,0.0,98,1.0,13585,Nathan Bock,0.0
2,2010,Fremantle,118,Adelaide,62,11712,Adelaide,8.0,4.0,13.0,...,0.0,0.0,1.0,1.0,0.0,78,1.0,13585,Myke Cook,0.0
3,2010,Fremantle,118,Adelaide,62,11700,Adelaide,8.0,3.0,12.0,...,0.0,1.0,2.0,5.0,0.0,82,1.0,13585,Patrick Dangerfield,0.0
4,2010,Fremantle,118,Adelaide,62,85,Adelaide,4.0,3.0,13.0,...,0.0,0.0,4.0,0.0,0.0,91,1.0,13585,Michael Doughty,0.0
5,2010,Fremantle,118,Adelaide,62,2381,Adelaide,2.0,3.0,10.0,...,0.0,0.0,1.0,1.0,0.0,74,1.0,13585,Richard Douglas,0.0
6,2010,Fremantle,118,Adelaide,62,34,Adelaide,16.0,8.0,9.0,...,1.0,0.0,1.0,2.0,0.0,83,1.0,13585,Tyson Edwards,0.0
7,2010,Fremantle,118,Adelaide,62,51,Adelaide,7.0,2.0,17.0,...,0.0,0.0,2.0,0.0,0.0,91,1.0,13585,Simon Goodwin,0.0
8,2010,Fremantle,118,Adelaide,62,11534,Adelaide,6.0,3.0,2.0,...,1.0,0.0,1.0,0.0,1.0,72,1.0,13585,Jonathon Griffin,0.0
9,2010,Fremantle,118,Adelaide,62,1434,Adelaide,3.0,2.0,11.0,...,0.0,0.0,1.0,0.0,0.0,78,1.0,13585,Trent Hentschel,0.0


In [88]:
rolling_groups = (player_df[['player_id', 'kicks', 'marks', 'handballs', 'goals', 'behinds', 'hit_outs', 'tackles',
                                   'rebounds', 'inside_50s', 'clearances', 'clangers', 'frees_for',
                                   'frees_against', 'contested_possessions',
                                   'uncontested_possessions', 'contested_marks', 'marks_inside_50',
                                   'one_percenters', 'bounces', 'goal_assists', 'time_on_ground']]
                        .sort_index()
                        .groupby('player_id', group_keys=False))

rolling_stats = rolling_groups.rolling(window=23).mean()

# Only select rows that are NaNs in rolling series
blank_rolling_rows = rolling_stats.isna()
expanding_stats = rolling_groups.expanding(1).mean()[blank_rolling_rows]

expanding_rolling_stats = (
    pd
    .concat([rolling_stats, expanding_stats], join='inner')
    .dropna()
    .sort_index()
    .drop('player_id', axis=1)
)

expanding_rolling_stats

,kicks,marks,handballs,goals,behinds,hit_outs,tackles,rebounds,inside_50s,clearances,...,frees_for,frees_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,time_on_ground
id,,,,,,,,,,,,,,,,,,,,,
101213581,11.000000,6.000000,5.000000,3.000000,1.000000,0.000000,1.000000,0.000000,3.000000,3.000000,...,2.000000,0.000000,6.000000,8.000000,1.000000,3.000000,0.000000,0.000000,4.000000,100.000000
101213590,12.500000,8.000000,6.000000,2.000000,0.500000,0.000000,1.500000,0.000000,4.000000,1.500000,...,1.000000,0.000000,5.000000,12.500000,0.500000,2.500000,0.500000,0.000000,3.000000,95.500000
101213598,13.666667,9.333333,4.666667,2.000000,0.666667,0.000000,1.666667,0.000000,6.000000,1.000000,...,0.666667,0.333333,5.000000,12.666667,1.000000,2.333333,0.333333,0.333333,2.666667,95.000000
101213603,13.500000,8.000000,5.500000,2.000000,0.750000,0.250000,2.000000,0.000000,5.750000,1.500000,...,0.500000,0.500000,4.750000,13.500000,0.750000,1.750000,0.500000,0.250000,2.250000,93.250000
101213611,13.400000,7.600000,6.000000,2.000000,0.600000,0.200000,2.200000,0.200000,5.600000,1.800000,...,0.800000,0.800000,5.200000,13.800000,0.800000,1.400000,0.800000,0.200000,1.800000,94.000000
101213622,13.166667,7.333333,5.500000,2.166667,1.000000,0.166667,2.166667,0.166667,5.500000,1.500000,...,0.666667,1.166667,4.666667,13.666667,0.666667,1.333333,1.000000,0.166667,1.500000,95.000000
101213631,12.571429,7.000000,5.714286,2.000000,1.000000,0.142857,2.285714,0.285714,5.000000,1.571429,...,0.571429,1.571429,4.857143,13.428571,0.571429,1.428571,1.000000,0.285714,1.285714,94.571429
101213635,12.750000,7.125000,5.750000,2.250000,1.000000,0.250000,2.125000,0.250000,5.375000,1.750000,...,0.625000,1.625000,5.375000,13.000000,0.625000,1.500000,0.875000,0.250000,1.125000,95.250000
101213644,12.777778,7.222222,6.222222,2.000000,1.000000,0.222222,2.333333,0.333333,5.111111,1.777778,...,0.777778,1.444444,6.333333,12.444444,0.666667,1.444444,0.888889,0.222222,1.222222,95.555556


In [89]:
brownlow_df = (player_df[['year', 'home_team', 'home_score', 'away_team', 'away_score', 'player_id', 'playing_for',
                       'round_number', 'match_id', 'player_name']]
            .merge(brownlow_last_year, on=['player_id', 'year'], how='left')
            .set_index(player_df['player_id'].astype(str) + player_df['match_id'].astype(str)))
games_played_df = player_df.sort_index().groupby('player_id').cumcount()

player_stats = pd.concat([brownlow_df, expanding_rolling_stats, games_played_df], axis=1, sort=True)
player_stats

,year,home_team,home_score,away_team,away_score,player_id,playing_for,round_number,match_id,player_name,...,frees_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,time_on_ground,0
101213581,2010,Sydney,88,St Kilda,96,1012,Sydney,1.0,13581,Adam Goodes,...,0.000000,6.000000,8.000000,1.000000,3.000000,0.000000,0.000000,4.000000,100.000000,0
101213590,2010,Adelaide,75,Sydney,118,1012,Sydney,2.0,13590,Adam Goodes,...,0.000000,5.000000,12.500000,0.500000,2.500000,0.500000,0.000000,3.000000,95.500000,1
101213598,2010,Sydney,111,Richmond,56,1012,Sydney,3.0,13598,Adam Goodes,...,0.333333,5.000000,12.666667,1.000000,2.333333,0.333333,0.333333,2.666667,95.000000,2
101213603,2010,North Melbourne,60,Sydney,100,1012,Sydney,4.0,13603,Adam Goodes,...,0.500000,4.750000,13.500000,0.750000,1.750000,0.500000,0.250000,2.250000,93.250000,3
101213611,2010,Sydney,115,West Coast,63,1012,Sydney,5.0,13611,Adam Goodes,...,0.800000,5.200000,13.800000,0.800000,1.400000,0.800000,0.200000,1.800000,94.000000,4
101213622,2010,Sydney,107,Brisbane Lions,87,1012,Sydney,6.0,13622,Adam Goodes,...,1.166667,4.666667,13.666667,0.666667,1.333333,1.000000,0.166667,1.500000,95.000000,5
101213631,2010,Geelong,126,Sydney,59,1012,Sydney,7.0,13631,Adam Goodes,...,1.571429,4.857143,13.428571,0.571429,1.428571,1.000000,0.285714,1.285714,94.571429,6
101213635,2010,Western Bulldogs,101,Sydney,63,1012,Sydney,8.0,13635,Adam Goodes,...,1.625000,5.375000,13.000000,0.625000,1.500000,0.875000,0.250000,1.125000,95.250000,7
101213644,2010,Sydney,63,Fremantle,100,1012,Sydney,9.0,13644,Adam Goodes,...,1.444444,6.333333,12.444444,0.666667,1.444444,0.888889,0.222222,1.222222,95.555556,8
101213655,2010,Hawthorn,79,Sydney,77,1012,Sydney,10.0,13655,Adam Goodes,...,1.500000,6.400000,11.900000,0.900000,1.800000,0.800000,0.200000,1.300000,95.800000,9


In [90]:
home_stats = (player_stats
              [player_stats['playing_for'] == player_stats['home_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('home_', '').replace('away_', 'oppo_'))
              .assign(at_home=1))
away_stats = (player_stats
              [player_stats['playing_for'] == player_stats['away_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('away_', '').replace('home_', 'oppo_'))
              .assign(at_home=0))
team_df = pd.concat([home_stats, away_stats], sort=True).sort_index()

team_df

,at_home,behinds,bounces,clangers,clearances,contested_marks,contested_possessions,frees_against,frees_for,goal_assists,...,player_name,playing_for,rebounds,round_number,score,tackles,team,time_on_ground,uncontested_possessions,year
101213581,1,1.000000,0.521739,2.130435,1.478261,1.086957,6.434783,0.608696,0.956522,1.043478,...,Adam Goodes,Sydney,0.391304,1.0,88,2.130435,Sydney,87.695652,9.782609,2010
101213590,0,0.000000,0.000000,2.000000,0.000000,0.000000,4.000000,0.000000,0.000000,2.000000,...,Adam Goodes,Sydney,0.000000,2.0,118,2.000000,Sydney,91.000000,17.000000,2010
101213598,1,0.913043,0.478261,1.956522,1.304348,1.086957,6.565217,0.608696,0.652174,0.956522,...,Adam Goodes,Sydney,0.434783,3.0,111,2.304348,Sydney,82.913043,10.130435,2010
101213603,0,1.130435,0.304348,2.043478,1.478261,1.304348,7.173913,0.826087,0.695652,0.913043,...,Adam Goodes,Sydney,0.478261,4.0,100,2.434783,Sydney,85.260870,9.913043,2010
101213611,1,1.000000,0.478261,2.173913,1.869565,0.956522,6.391304,0.782609,0.956522,0.739130,...,Adam Goodes,Sydney,0.391304,5.0,115,2.304348,Sydney,87.739130,8.869565,2010
101213622,1,1.130435,0.478261,2.304348,1.782609,0.826087,6.130435,0.826087,0.913043,0.739130,...,Adam Goodes,Sydney,0.391304,6.0,107,2.391304,Sydney,87.739130,9.043478,2010
101213631,0,1.304348,0.304348,2.869565,2.347826,1.173913,8.652174,0.913043,0.913043,0.695652,...,Adam Goodes,Sydney,0.739130,7.0,59,3.173913,Sydney,89.826087,10.347826,2010
101213635,0,1.130435,0.173913,2.695652,2.086957,1.565217,8.434783,0.956522,0.913043,0.478261,...,Adam Goodes,Sydney,0.521739,8.0,63,2.434783,Sydney,90.652174,9.782609,2010
101213644,1,1.260870,0.217391,3.086957,2.434783,1.391304,8.782609,0.869565,1.086957,0.782609,...,Adam Goodes,Sydney,0.739130,9.0,63,2.956522,Sydney,90.086957,10.391304,2010
101213655,0,1.260870,0.347826,2.608696,2.217391,0.956522,8.521739,0.913043,0.913043,1.000000,...,Adam Goodes,Sydney,0.565217,10.0,77,3.173913,Sydney,89.913043,9.826087,2010


In [99]:
any(team_df.index.duplicated())

False

In [106]:
(pd.concat([team_df.groupby(['team', 'year', 'round_number']).sum()[['kicks', 'marks', 'handballs', 'goals', 'behinds', 'hit_outs', 'tackles',
                                   'rebounds', 'inside_50s', 'clearances', 'clangers', 'frees_for',
                                   'frees_against', 'contested_possessions',
                                   'uncontested_possessions', 'contested_marks', 'marks_inside_50',
                                   'one_percenters', 'bounces', 'goal_assists', 'time_on_ground']],
            team_df[['at_home', 'round_number', 'score', 'year', 'team', 'oppo_team', 'oppo_score']]
             .set_index(['team', 'year', 'round_number'], drop=False)
             .drop_duplicates(subset=['team', 'year', 'round_number'], keep='last')], axis=1, sort=True))

kicks       marks   handballs  \
team             year round_number                                       
Adelaide         2010 1.0           157.000000   73.000000  194.000000   
                      2.0           169.500000   82.500000  173.500000   
                      3.0           190.666667   97.000000  187.166667   
                      4.0           184.750000   96.000000  190.333333   
                      5.0           173.950000   94.533333  200.583333   
                      6.0           183.500000   99.083333  185.333333   
                      7.0           186.514286   99.811905  178.530952   
                      8.0           190.871429  110.490476  169.057143   
                      9.0           180.591270   95.005159  176.215079   
                      10.0          176.002778   95.385714  172.016270   
                      11.0          187.667749   97.026046  174.071681   
                      12.0          183.319949  103.291414  173.634596   
                      13.0          192.628510  103.600955  182.071639   
                      14.0          189.906077  102.352081  174.026032   
                      15.0          192.231824  101.020166  173.105270   
                      16.0          196.890568   99.539423  178.135256   
                      17.0          194.539289  100.341209  172.046350   
                      18.0          188.529844   98.748410  168.912100   
                      19.0          198.842179  102.551423  178.024923   
                      20.0          194.312151  101.057254  180.346189   
                      21.0          178.231665   96.801117  164.575654   
                      22.0          186.544203   98.836339  171.554360   
                 2011 1.0           200.909314  108.074345  172.723101   
                      3.0           199.420190  105.474757  164.609899   
                      4.0           188.551114   92.739837  167.856270   
                      5.0           194.716588   97.568440  170.202601   
                      6.0           192.248011   98.565061  159.880392   
                      7.0           189.638387   95.820923  159.163518   
                      8.0           187.991967   96.721662  159.874059   
                      9.0           187.531449   94.883271  157.346260   
...                                        ...         ...         ...   
Western Bulldogs 2017 16.0          200.839130   77.958696  188.213043   
                      17.0          209.131470   83.381470  195.857143   
                      18.0          209.330952   87.918364  195.228793   
                      19.0          213.768116   89.018979  190.399241   
                      20.0          208.804424   89.161747  183.514531   
                      21.0          209.946047   89.360227  182.529051   
                      22.0          210.781330   86.386189  182.235720   
                      23.0          210.755694   85.344030  197.157005   
                 2018 1.0           193.272311   77.845156  170.178490   
                      2.0           193.786957   84.334783  174.008696   
                      3.0           204.874534   91.190683  178.054244   
                      4.0           210.666008   91.389987  179.027997   
                      5.0           192.894203   86.046377  177.489855   
                      6.0           204.926812   90.184058  182.260145   
                      7.0           199.148464   88.589976  180.926587   
                      8.0           204.194846   86.307138  181.082315   
                      9.0           203.973430   82.553744  180.026570   
                      10.0          201.579503   86.990821  167.991097   
                      11.0          196.278162   84.550000  172.001680   
                      13.0          199.424177   83.260870  173.926394   
                      14.0          189.278457   84.773465  164.778908   
                      15.0          189.67376